# 장르 속성을 이용한 영화 콘텐츠 기반 필터링

## 데이터 로딩 및 가공
- TMDB 5000 데이터 셋: 영화 데이터 정보 사이트인 imdb.com의 영화 중 주요 영화 5,000개에 대한 메타 정보를 가공해서 kaggle에서 제공하는 데이터 세트
- https://www.kaggle.com/tmdb/tmdb-movie-metadata

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

movies = pd.read_csv('./dataset/tmdb_5000_movies.csv')
print(movies.shape)
display(movies.head())

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


- 분석에 사용할 주요 컬럼 추출
  - id, title, genres, vote_average(평균 평점), vote_count(평점 투표수), popularity(영화 인기도), keyword, overview(영화 개요)

In [2]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [5]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


In [3]:
pd.set_option('max_colwidth', 200)

In [10]:
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name..."


In [11]:
movies_df['genres'].values[0] # 리스트 형태이지만 데이터 유형은 문자타입

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

- eval()과 literal_eval()
  - eval()은 문자형태로 되어있는 표현식을 실행하는 함수로, 함수나 객체도 실행 가능
  - literal_eval()은 eval()과는 다르게 파이썬에서 제공하는 기본 데이터 타입 정도만 변환해주는 용도로 사용 가능

- literal_eval() 함수를 통해 genres,keywords 컬럼의 값을 리스트 객체로 변환

In [4]:
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [13]:
movies_df['genres'].values[0]  # 문자열이 아닌 리스트 객체로 바뀜

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

- 장르와 키워드 컬럼의 name 키의 값만 원소로 추출하여 리스트로 생성

In [5]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x ])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x ])

In [6]:
movies_df['genres'].values[0]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

## 장르 콘텐츠 유사도 측정
- 리스트로 변환된 장르 컬럼은 카운트 기반으로 피처 벡터화 변환
- sklearn의 CounterVectorizer 이용
- 장르 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교한다
- 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천한다
- 피처벡터화: 각 영화별 장르에 해당하는 것을 하나씩 꺼내 희소벡터로 변환(본인에 해당하는 장르는 1, 아닌것은 0으로) -> 희소 행렬로

[참고] CountVectorizer
- 텍스트에서 단위(단어)별 출현 횟수를 카운팅하여 수치 벡터화한다

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,1)) 
# ngram_range: 모델의 단어 순서를 어느 정도 보강하기 위한 범위, (범위 최소값, 범위 최대값)
# (1,1): 단어를 한개씩 피처로 추출
# (1,2): 토큰화된 단어를 1개씩 피처로 추출하고 또 순서대로 2개씩 묶어서 피처로 추출
# 한글처럼 두개의 단어가 붙어서 새로운 의미를 만들어낼 때 사용 
vectorizer.fit(['첫번째 문서 테스트', '두번째 문서 테스트']) # 4개의 어휘를 학습한 countervectorizer 생성
print(vectorizer.vocabulary_) # 고유한 단어가 각각의 인덱스를 가지게 됨
counts = vectorizer.transform(['직접 첫번째 테스트 두번째 테스트']) # 새로운 문서에 대해 미리 학습해놓은 사전을 기반으로 단어의 빈도수를 세어줌
print(counts)  # 밀집행렬의 형태
print('두번쨰:0, 문서:1, 첫번째:2, 테스트:3')
print(counts.toarray()) 

{'첫번째': 2, '문서': 1, '테스트': 3, '두번째': 0}
  (0, 0)	1
  (0, 2)	1
  (0, 3)	2
두번쨰:0, 문서:1, 첫번째:2, 테스트:3
[[1 0 1 2]]


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
# CounterVectorizer를 적용하기 위해 리스트가 아닌 공백 문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : ' '.join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
# min_df: 전체 문서에서 낮은 빈도수를 차지하는 문서를 제외
# min_df=0: 빈도수가 0이하인건 제외
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)
print(genre_mat.toarray()[:1])
# 영화가 4803개, 장르가 276개 (행: 영화, 열: 장르)
# 총 276개 장르에서 첫번째 영화가 해당하는 장르는 1의 값

(4803, 276)
[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [9]:
# 코사인 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat) # (기준행, 비교대상행)
print(genre_sim.shape)
print(genre_sim[:3])
# 4803개의 상호간 유사도

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]]


In [12]:
# 행기준 내림차순 정렬하고 인덱스값 반환하는 유사도 행렬
genre_sim_sorted_idx = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_idx[:1]) # 첫번째 영화와 유사도가 높은 영화의 인덱스값
print(genre_sim_sorted_idx[1])
print(genre_sim_sorted_idx[2])
print(genre_sim_sorted_idx[3])

[[   0 3494  813 ... 3038 3037 2401]]
[ 262    1  129 ... 3069 3067 2401]
[   2 1740 1542 ... 3000 2999 2401]
[2195 1850 3316 ...  887 2544 4802]


In [15]:
# 장르가 같은 영화는 유사도가 모두 1로 계산됨
print(movies_df.loc[262,'genres'])
print(movies_df.loc[1,'genres'])
print(movies_df.loc[129,'genres'])

['Adventure', 'Fantasy', 'Action']
['Adventure', 'Fantasy', 'Action']
['Adventure', 'Fantasy', 'Action']


## 장르 콘텐츠 필터링을 이용한 영화 추천

In [32]:
movies_df[movies_df['title'] == 'The Dark Knight Rises']

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.31295,"[dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham city, vigilante, cover-up, superhero, villainess, tragic hero, terrorism, destruction, catwoman, ca...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police...",Action Crime Drama Thriller


In [18]:
#(영화정보를 답담고있는 데이터 프레임, 유사도가 높은 순으로 정렬된 인덱스, 기준이 되는 영화, 추천 영화 개수)
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    # 기준영화의 영화 정보 가져오기
    target_movie = df[df['title'] == title_name]
    
    # 기준영화의 인덱스값 가져오기
    title_index = target_movie.index.values
    
    # 유사도가 높은순으로 정렬된 유사도 행렬에서 타겟 영화 행에서 추천 영화 개수만큼 슬라이싱해서 가져오기
    similar_index = sorted_idx[title_index, :top_n]
    #print(similar_index)
    
    # 추출된 top_n index가 2차원데이터이기 때문에 1차원 벡터로 변환
    similar_index = similar_index.reshape(-1) 

    return df.iloc[similar_index]

In [ ]:
- 좀 더 많은 후보군을 선정한 뒤에 **영화의 평점에 따라 필터링**하는 방식으로 보완 필요

In [19]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_idx, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


- 영화 평점이 높은 순으로 정렬하여 상위 10개 영화 확인

In [20]:
movies_df[['title','vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 평점에 평가 횟수를 반영한 가중 평점 방식 적용 필요
- 가중평점 = (v/(v+m)) * R + (m/(v+m)) * C
  - v: 개별 영화에 평점을 투표한 횟수
  - m: 평점을 부여하기 위한 최소 투표 횟수(임의값)
  - R: 개별 영화에 대한 평균 평점
  - C: 전체 영화에 대한 평균 평점

In [21]:
C = movies_df['vote_average'].mean()
# 전체 횟수에서 상위 60% 해당하는 횟수를 기준으로 정함
m = movies_df['vote_count'].quantile(0.6)
print(f'C: {C:.3f}, m: {m:.3f}')

C: 6.092, m: 370.200


In [22]:
# 가중평점 계산 함수
def weighted_vote_average(movie):
    v = movie['vote_count']
    R = movie['vote_average']

    return (v/(v+m)) * R + (m/(v+m)) * C

# 가중평점 컬럼 추가
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [23]:
movies_df[['title','vote_average', 'weighted_vote','vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [26]:
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    # 기준영화의 영화 정보 가져오기
    target_movie = df[df['title'] == title_name]
    
    # 기준영화의 인덱스값 가져오기
    title_index = target_movie.index.values
    
    # 장르 유사도가 높은 인덱스를 추천 개수의 2배수만큼 추출
    similar_index = sorted_idx[title_index, :top_n*2]
    
    # 추출된 top_n index가 2차원데이터이기 때문에 1차원 벡터로 변환
    similar_index = similar_index.reshape(-1) 

    # 기준 영화 index는 제외
    similar_index = similar_index[similar_index != title_index]
    
    # 가중평점으로 내림차순 정렬 후 추천 개수만큼 추출
    return df.iloc[similar_index].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_idx, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
